### Photovoltaic motif Discovery

In [23]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import stumpy

import DataRetriever as dr

retriever = dr.DataRetriever()
hour = retriever.get_data("All-Subsystems-hour-Year2.pkl")

In [24]:
gen_df = hour[hour["PV_Watts3PhTotalW3PhT1"].isna() == False] #There are 87 rows with NaN values.
gen_df["PV_Watts3PhTotalW3PhT1"] = gen_df["PV_Watts3PhTotalW3PhT1"] / 1e3 #Set the Wh to kWh
gen_df

C:\Users\magnu\AppData\Local\Temp\ipykernel_7084\2653879736.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Timestamp,Load_LatentHeatWaterVolume,Load_RefrigeratorTemp,Load_StatusBA1Lights,Load_StatusKitchenLightsA,Load_StatusKitchenLightsB,Load_StatusKitchenLightsC,Load_StatusDRLights,Load_StatusLRLights3,Load_StatusEntryHallLights,...,SHW_GlycolFlowHXCoriolisSHW,SHW_WaterFlowHXCoriolisSHW,SHW_GlycolFlowRateHXCoriolisSHW,SHW_WaterFlowRateHXCoriolisSHW,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierAirflow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2015-02-01 00:00:00,2015-02-01 00:00:00,0.060105,4.957915,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.449169,1045.817746,4.498593,72.228102,74.959034,0.0
2015-02-01 01:00:00,2015-02-01 01:00:00,0.131439,4.904702,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,391.606267,1042.026917,4.442783,71.963450,73.979483,0.0
2015-02-01 02:00:00,2015-02-01 02:00:00,0.194187,5.054887,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,388.484683,1023.145217,4.444083,71.921017,73.818717,0.0
2015-02-01 03:00:00,2015-02-01 03:00:00,0.258255,4.991525,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,175.792833,1066.177350,4.467983,71.844683,74.535050,0.0
2015-02-01 04:00:00,2015-02-01 04:00:00,0.322324,5.040868,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,NaN,NaN,NaN,NaN,371.120983,1081.112783,4.466550,71.959983,74.741467,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-31 19:00:00,2016-01-31 19:00:00,1.472254,4.810046,0.0,0.750000,0.750000,0.750000,0.766667,0.750000,0.0,...,419.930636,942.630328,0.001339,0.000336,11.229733,22.489150,4.243500,67.764183,68.643883,0.0
2016-01-31 20:00:00,2016-01-31 20:00:00,1.546230,4.917617,0.0,0.750000,0.750000,0.750000,0.483333,1.000000,0.0,...,419.930636,942.630328,0.001503,0.000385,11.366567,22.483300,4.275667,67.889300,68.772900,0.0
2016-01-31 21:00:00,2016-01-31 21:00:00,1.617564,4.820712,0.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.0,...,419.930636,942.630328,0.001470,0.000437,11.448617,22.509617,4.297500,67.785950,68.651417,0.0


#### Calculating the matrix profile and basic statistics

In [25]:
m = 24*7 #Set the window size to a week since we have hourly timestamps
mp = stumpy.stump(gen_df["PV_Watts3PhTotalW3PhT1"], m=m)
#Array 0 is distances, array 1 is nearest neighbor, array 2 is nearest neighbor to the left and array 3 is nearest neighbor to the right.
mp

array([[5.027679994938788, 6359, -1, 6359],
       [5.0276803494568325, 6360, -1, 6360],
       [5.027682060367196, 6361, -1, 6361],
       ...,
       [4.97035699278659, 7197, 7197, -1],
       [4.97035347039983, 7198, 7198, -1],
       [4.970350820351731, 7199, 7199, -1]], dtype=object)

In [26]:
mp[:, 0] #Accessing a column in the matrix profile, in this case the distances.

array([5.027679994938788, 5.0276803494568325, 5.027682060367196, ...,
       4.97035699278659, 4.97035347039983, 4.970350820351731],
      dtype=object)

In [27]:
mp_median_distance = np.median(mp[:, 0])

print(f"The median nearest neighbor distance between sequences is {mp_median_distance}")

mp_mean_distance = np.mean(mp[:, 0])

print(f"The mean nearest neighbor distance between sequences is {mp_mean_distance}")

mp_quantile_distance = np.quantile(mp[:, 0], q=0.25)

print(f"The lower quartile for distance between nearest neighbor sequences is {mp_quantile_distance}")

The median nearest neighbor distance between sequences is 4.7207997178190375
The mean nearest neighbor distance between sequences is 4.811202683792316
The lower quartile for distance between nearest neighbor sequences is 4.067082661001723


#### Finding global motif, its neareest neighbor and visualising it

In [28]:
global_motif_idx = np.argsort(mp[:, 0])[0] # Takes all distances in column 0, and sort them in ascending order, this returns the indexes. Then take the first index in sorted array.

print(f"The global motif is located at index {global_motif_idx}. Global motif is the subsequence with smallest distance to its nearest neighbor.")

nearest_neighbor_idx = mp[global_motif_idx, 1] #Access the previous index, and return the index of its nearest neighbor.

print(f"The nearest neighbor to the global motif is located at index {nearest_neighbor_idx}")

The global motif is located at index 2635. Global motif is the subsequence with smallest distance to its nearest neighbor.
The nearest neighbor to the global motif is located at index 4099


In [29]:
fig = make_subplots(rows=3, cols=2,
                    specs=[
                        [{"colspan": 2}, None],
                        [{"colspan": 2}, None],
                        [{}, {}]
                    ],
                    subplot_titles=(" ", "Matrix Profile", "First Occurrence", "Second Occurrence")
                    )

fig.add_trace(
    go.Scatter(x=gen_df['Timestamp'], y=gen_df["PV_Watts3PhTotalW3PhT1"], name="Power Generated"),
    row=1, col=1)

fig.add_trace(
    go.Scatter(y=mp[:, 0], name="Distance Matrix"),
    row=2, col=1)

fig.add_trace(
    go.Scatter(x=gen_df[global_motif_idx: global_motif_idx+m]['Timestamp'], y=gen_df[global_motif_idx: global_motif_idx+m]["PV_Watts3PhTotalW3PhT1"], name="Zoomed in view"),
    row=3, col=1)

fig.add_trace(
    go.Scatter(x=gen_df[nearest_neighbor_idx: nearest_neighbor_idx+m]['Timestamp'], y=gen_df[nearest_neighbor_idx: nearest_neighbor_idx+m]["PV_Watts3PhTotalW3PhT1"], name="Zoomed in view"),
    row=3, col=2)

fig.update_yaxes(title_text="Energy Generated [kWh]", title_font=dict(size=12),  range=[0, 5], row=1, col=1)
fig.update_yaxes(title_text="Euclidean Distance", title_font=dict(size=12), range=[0, 12], row=2, col=1)

fig.update_layout(
    shapes=[
        dict(type="rect", xref="x1", yref="y1",
             x0=gen_df["Timestamp"][global_motif_idx], x1=gen_df["Timestamp"][global_motif_idx+m], y0=0, y1=5),
        dict(type="rect", xref="x1", yref="y1",
             x0=gen_df["Timestamp"][nearest_neighbor_idx], x1=gen_df["Timestamp"][nearest_neighbor_idx+m], y0=0, y1=5),
        dict(type="line", xref="x2", yref="y2",
             x0=global_motif_idx, x1=global_motif_idx, y0=0, y1=12, line=dict(dash="dot")),
        dict(type="line", xref="x2", yref="y2",
             x0=nearest_neighbor_idx, x1=nearest_neighbor_idx, y0=0, y1=12, line=dict(dash="dot"))
    ],
    height=700, showlegend=False)

fig.show()

#### Counting all subsequences in the global motif.

In [30]:
#match takes a query as input and matches that subsequence to all other subsequences in the time series.
global_motif_count = stumpy.match(
    Q = gen_df["PV_Watts3PhTotalW3PhT1"][global_motif_idx: global_motif_idx+m],
    T = gen_df["PV_Watts3PhTotalW3PhT1"],
    max_distance = mp_mean_distance
)

print(f"max_distance allowed between subsequences is set to {mp_mean_distance}")

global_motif_count

max_distance allowed between subsequences is set to 4.811202683792316


array([[4.3187668163148874e-07, 2635],
       [2.2825339420155735, 4099],
       [3.4765270418642875, 4627],
       [3.6724151870739066, 3043],
       [3.7332845748469037, 4843],
       [4.015492119797373, 4579],
       [4.129827627217115, 2155],
       [4.189481215100035, 4939],
       [4.223372893037016, 4363],
       [4.495029184608503, 4987],
       [4.533827111674303, 5179],
       [4.592538397287818, 2683],
       [4.635634004990765, 3331],
       [4.736425665472354, 2995],
       [4.753860432659059, 2011],
       [4.7608501070962665, 4291],
       [4.80231690036927, 1795]], dtype=object)

In [31]:
fig = go.Figure()

title = f"All {len(global_motif_count)} subsequences in the global motif"

for idx in global_motif_count[:, 1]:
    fig.add_trace(go.Scatter(
        y = gen_df[idx: idx+m]["PV_Watts3PhTotalW3PhT1"],
        marker = dict(color="rgba(98, 111, 250, 0.3)"),
    ))

fig.update_layout(showlegend=False,
                  title={'text' : title,
                         'xanchor': 'center', 'x' : 0.5,
                         'y' : 0.83, 'yanchor' : 'top'})
fig.update_yaxes(title="Energy Generated [kWh]")
fig.update_xaxes(title="", visible=False)
fig.show()

#### Finding all top motifs

In [35]:
#.motifs returns all motifs in the timeseries.
top_motifs = stumpy.motifs(
    T = gen_df["PV_Watts3PhTotalW3PhT1"],
    P = mp[:, 0], #Matrix profile
    max_distance=mp_mean_distance, #max allowed distance between candidate sequence Q and non-trivial sequence S to be a match.
    min_neighbors=5.0, #min number of similar matches candidate sequence Q has to have to be considered a motif.
    max_matches=50, #max amount of similar subsequences to Q to return.
    max_motifs=50, #max amount of motifs to return.
    cutoff=mp_mean_distance #the largest matrix profile distance a candidate sequence Q can have before not being a motif.
    )

print(mp_mean_distance)

top_motifs

4.811202683792316


c:\users\magnu\appdata\local\programs\python\python39\lib\site-packages\stumpy\motifs.py:280: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



(array([[4.31876682e-07, 2.28253394e+00, 3.47652704e+00, 3.67241519e+00,
         3.73328457e+00, 4.01549212e+00, 4.12982763e+00, 4.18948122e+00,
         4.22337289e+00, 4.49502918e+00, 4.53382711e+00, 4.59253840e+00,
         4.63563400e+00, 4.73642567e+00, 4.75386043e+00, 4.76085011e+00,
         4.80231690e+00],
        [5.11003381e-07, 2.97450505e+00, 3.37470593e+00, 4.19107739e+00,
         4.34361858e+00, 4.50308447e+00, 4.54966863e+00, 4.61436790e+00,
                    nan,            nan,            nan,            nan,
                    nan,            nan,            nan,            nan,
                    nan],
        [9.05912170e-07, 3.32276309e+00, 3.96848153e+00, 4.25013871e+00,
         4.41325414e+00, 4.48952370e+00, 4.53648067e+00, 4.61815084e+00,
         4.65773178e+00, 4.77028134e+00,            nan,            nan,
                    nan,            nan,            nan,            nan,
                    nan],
        [9.05912170e-07, 3.66372691e+00, 3.860

In [36]:
top_motifs[1] #Accessing indexes for all motifs.

array([[2635, 4099, 4627, 3043, 4843, 4579, 2155, 4939, 4363, 4987, 5179,
        2683, 3331, 2995, 2011, 4291, 1795],
       [2578, 4522, 4042, 1690, 5410, 4762, 2722, 1954,   -1,   -1,   -1,
          -1,   -1,   -1,   -1,   -1,   -1],
       [5491, 1651, 5371, 4387, 2539, 1387, 3907, 6019, 5923, 2683,   -1,
          -1,   -1,   -1,   -1,   -1,   -1],
       [4892, 4820, 4604, 2156, 3020, 2684, 3716, 4316, 1772, 4028, 4964,
          -1,   -1,   -1,   -1,   -1,   -1],
       [5076, 1764, 2988, 5916, 2148, 4860,   -1,   -1,   -1,   -1,   -1,
          -1,   -1,   -1,   -1,   -1,   -1]], dtype=int64)

In [37]:
top_motifs[1][0] #Accessing a single motif.

array([2635, 4099, 4627, 3043, 4843, 4579, 2155, 4939, 4363, 4987, 5179,
       2683, 3331, 2995, 2011, 4291, 1795], dtype=int64)

In [ ]:
ordinal = lambda n: "%d%s" % (n, "tsnrhtdd"[(n // 10 % 10 != 1) * (n % 10 < 4) * n % 10::4]) #Dont ask
motif_names = [ordinal(n) + " Motif" for n in range(1, 1+len(top_motifs[1]))]
for i in range(0, len(top_motifs[1])):
    motif_names[i] = motif_names[i] + f" ({len([entry for entry in top_motifs[1][i] if entry != -1])} Occurrences)"

fig = make_subplots(rows=len(top_motifs[1]), cols=1,
                    y_title="Energy Generated [kWh]",
                    subplot_titles=(motif_names),
                    shared_xaxes=True,)
row = 1
col = 1

for i in top_motifs[1]:
    for j in i:
        fig.add_trace(go.Scatter(x=gen_df[j: j+m].reset_index(drop=True).index, y=gen_df[j: j+m]["PV_Watts3PhTotalW3PhT1"], mode="lines",
                             line=dict(color=f"rgba(98, 111, 250, {2/len(i)})")), row=row, col=col)
    row += 1

fig.update_layout(showlegend=False, height=600)
fig.update_annotations(font_size=12)
fig.update_xaxes(visible=False)


fig.show()

#### Visualising the PV discords

In [38]:
discord_indexes = []
copy_mp = mp.copy()

#We want to return top three potential discords. However, we want non-trivial subsequences, and so we implement a way to return top distance indexes with regards to the exclusion zone m//4.
while True:
    discord_distance = np.amax(copy_mp[:, 0]) #amax returns the value of the largest distance in the matrix profile
    discord_index = np.argmax(copy_mp[:, 0]) #argmax returns the index of the largest distance in the matrix profile.

    discord_indexes.append(np.where(mp[:, 0] == discord_distance)) #We append the original index from mp to our top 3 list.

    # We then delete from our copy_mp the indexes in the exclusion zone from the discord index. Next iteration thereby finds a new discord not trivial to the last discord sequence.
    copy_mp = np.delete(copy_mp, list(range(max(0, discord_index-m//4), min(len(copy_mp), discord_index+m//4+1))), axis=0)

    if len(discord_indexes) == 3:
        break

discord_indexes

[(array([8411], dtype=int64),),
 (array([372], dtype=int64),),
 (array([5769], dtype=int64),)]

In [ ]:
names = []
for i in discord_indexes:
    idx = i[0][0]
    dist = round(mp[:, 0][idx], 2)
    names.append(dist)

fig = make_subplots(rows=3, cols=1,
                    subplot_titles=[f"Anomaly 1 <br>Distance to NN: {names[0]}", f"Anomaly 2 <br>Distance to NN: {names[1]}",
                                    f"Anomaly 3 <br>Distance to NN: {names[2]}"]
                    )

row = 1
col = 1

for i in discord_indexes:
    idx = i[0][0]
    fig.add_trace(go.Scatter(
        x=gen_df[idx: idx+m].index,
        y=gen_df[idx: idx+m]["PV_Watts3PhTotalW3PhT1"],
        mode="lines"
    ), row=row, col=col)
    row += 1

fig.update_layout(height=600, showlegend=False)

fig.update_yaxes(title_text='Energy Generated [kWh]', title_font=dict(size=16), row=2, col=1)

fig.show()